# It is not recommended to run this on your machine !!!
you might get blocked by youtube
ไม่แนะนำให้รันในเครื่องของคุณเองอาจโดนบล็อกจาก youtube

In [1]:
!pip install yt-dlp librosa==0.9.2 piano_transcription_inference joblib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.2.post1
    Uninstalling librosa-0.1

In [2]:
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


# import

In [1]:
import pandas as pd

from piano_transcription_inference import PianoTranscription, sample_rate, load_audio
from joblib import Parallel, delayed
from tqdm import tqdm

import random
import tempfile
import os
import glob
import subprocess
import multiprocessing
import time

In [2]:
transcriptor = PianoTranscription(device='cuda')    # 'cuda' | 'cpu'

Checkpoint path: /home/bk/piano_transcription_inference_data/note_F1=0.9677_pedal_F1=0.9186.pth
Total size: ~165 MB


--2024-06-03 10:32:17--  https://zenodo.org/record/4034264/files/CRNN_note_F1%3D0.9677_pedal_F1%3D0.9186.pth?download=1
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.184.103.159, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/4034264/files/CRNN_note_F1%3D0.9677_pedal_F1%3D0.9186.pth [following]
--2024-06-03 10:32:19--  https://zenodo.org/records/4034264/files/CRNN_note_F1%3D0.9677_pedal_F1%3D0.9186.pth
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 171966578 (164M) [application/octet-stream]
Saving to: ‘/home/bk/piano_transcription_inference_data/note_F1=0.9677_pedal_F1=0.9186.pth’

     0K .......... .......... .......... .......... ..........  0% 96.8K 28m54s
    50K .......... .......... .......... .......... ..........  0% 11.8M 14m34s
   100K .......... .......... .......... .......... ......

Using cuda for inference.
GPU number: 1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# load dataset link

In [3]:
dataset_path = "song2midi/train_dataset.csv"
dataset = pd.read_csv(dataset_path)

In [5]:
load_from = 124
load_to = 1000 # (load_from to load_to - 1)

data_to_load = dataset[load_from:load_to]
data_to_load

,piano_ids,pop_ids
124,8iX4BB0Ti6A,8g-cF9H53lU
125,8lGI0grHjog,8cQFayyl_YQ
126,8pV6G0CGTKU,o6qY2xsP66k
127,93bc2uUGQR4,lUapkbzxQmM
128,9E9k1eoaHSY,0dZnf09g77g
...,...,...
995,359OxJP3fAQ,t1yrzSXU_Xw
996,36NUOCx65qA,8R57RnWSq84
997,36gJssa_Q3A,i2u5mJPraHA
998,36lfT7YNmd0,3uHDbEBE460


# initialize

In [7]:
os.makedirs("song2midi/midi_output", exist_ok=True)
os.makedirs("song2midi/piano_output", exist_ok=True)
os.makedirs("song2midi/song_output", exist_ok=True)

In [24]:
def download_piano(piano_id, song_id, output_path="piano_output"):
    piano_download_output = f"{output_path}/{song_id}___%(uploader)s___%(title)s___%(id)s___%(duration)d___.%(ext)s"

    result = subprocess.run(["yt-dlp",
                             "-o", piano_download_output,
                             "-x",
                             "--audio-quality", "0",
                             "--audio-format", "wav",
                             "--retries", "60",
                             "--force-ipv4",
                             "--ignore-errors",
                             f"https://www.youtube.com/watch?v={piano_id}"])

    return result

In [9]:
def download_midi(piano_id, song_id, output_path="song2midi/midi_output"):
    with tempfile.TemporaryDirectory() as tmpdir:
        piano_download_output = f"{tmpdir}/{song_id}___%(uploader)s___%(title)s___%(id)s___%(duration)d___.%(ext)s"

        result = subprocess.run(["yt-dlp",
                                 "-o", piano_download_output,
                                 "-x",
                                 "--audio-quality", "0",
                                 "--audio-format", "wav",
                                 "--retries", "60",
                                 "--force-ipv4",
                                 "--sleep-interval", "5",
                                 "--ignore-errors",
                                 f"https://www.youtube.com/watch?v={piano_id}"])

        for piano_wav in os.listdir(tmpdir):
            if "ipynb" in piano_wav:
                continue
            (audio, _) = load_audio(os.path.join(tmpdir, piano_wav), sr=16000, mono=True)
            transcriptor.transcribe(audio, os.path.join(output_path, f"{'___'.join(piano_wav.split('___')[:4])}.mid"))

    return result

In [26]:
def download_midi_by_link(piano_link, song_id, output_path="midi_output"):
    with tempfile.TemporaryDirectory() as tmpdir:
        piano_download_output = f"{tmpdir}/{song_id}___%(uploader)s___%(title)s___%(id)s___%(duration)d___.%(ext)s"

        result = subprocess.run(["yt-dlp",
                                 "-o", piano_download_output,
                                 "-x",
                                 "--audio-quality", "0",
                                 "--audio-format", "wav",
                                 "--retries", "60",
                                 "--force-ipv4",
                                 "--ignore-errors",
                                 piano_link])

        for piano_wav in os.listdir(tmpdir):
            if "ipynb" in piano_wav:
                continue
            (audio, _) = load_audio(os.path.join(tmpdir, piano_wav), sr=16000, mono=True)
            transcriptor.transcribe(audio, os.path.join(output_path, f"{'___'.join(piano_wav.split('___')[:4])}.mid"))

    return result

In [16]:
def download_song(song_id, output_path="song2midi/song_output"):
    song_download_output = f"{output_path}/%(id)s___%(uploader)s___%(title)s___%(duration)d___.%(ext)s"

    result = subprocess.run(["yt-dlp",
                               "-o", song_download_output,
                               "-x",
                               "--audio-quality", "5",
                               "--audio-format", "mp3",
                               "--retries", "50",
                               "sleep-interval", "7",
                               "--force-ipv4",
                               "--ignore-errors",
                               f"https://www.youtube.com/watch?v={song_id}"])

    return result

In [19]:
def download_song_by_link(song_link, output_path="song_output"):
    song_download_output = f"{output_path}/%(id)s___%(uploader)s___%(title)s___%(duration)d___.%(ext)s"
    time.sleep(random.randint(2, 7))

    result = subprocess.run(["yt-dlp",
                               "-o", song_download_output,
                               "-x",
                               "--audio-quality", "5",
                               "--audio-format", "mp3",
                               "--retries", "50",
                               "--force-ipv4",
                               "--ignore-errors",
                               song_link])

    return result

# testing

In [20]:
download_piano("-7lV0oJ0QXc", "EHl_eQhgefw")

CompletedProcess(args=['yt-dlp', '-o', 'piano_output/EHl_eQhgefw___%(uploader)s___%(title)s___%(id)s___%(duration)d___.%(ext)s', '-x', '--audio-quality', '0', '--audio-format', 'wav', '--retries', '50', '--force-ipv4', '--ignore-errors', 'https://www.youtube.com/watch?v=-7lV0oJ0QXc'], returncode=0)

In [21]:
download_midi("-7lV0oJ0QXc", "EHl_eQhgefw")

/usr/local/lib/python3.10/dist-packages/piano_transcription_inference/utilities.py:556: FutureWarning: Pass orig_sr=48000, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  y = librosa.core.audio.resample(y, sr_native, sr, res_type=res_type)


Segment 0 / 35
Segment 1 / 35
Segment 2 / 35
Segment 3 / 35
Segment 4 / 35
Segment 5 / 35
Segment 6 / 35
Segment 7 / 35
Segment 8 / 35
Segment 9 / 35
Segment 10 / 35
Segment 11 / 35
Segment 12 / 35
Segment 13 / 35
Segment 14 / 35
Segment 15 / 35
Segment 16 / 35
Segment 17 / 35
Segment 18 / 35
Segment 19 / 35
Segment 20 / 35
Segment 21 / 35
Segment 22 / 35
Segment 23 / 35
Segment 24 / 35
Segment 25 / 35
Segment 26 / 35
Segment 27 / 35
Segment 28 / 35
Segment 29 / 35
Segment 30 / 35
Segment 31 / 35
Segment 32 / 35
Segment 33 / 35
Segment 34 / 35
Segment 35 / 35
Write out to midi_output/EHl_eQhgefw___Montechait___The Beatles - With a Little Help from My Friends ｜ Piano Cover + Sheet Music___-7lV0oJ0QXc.mid


CompletedProcess(args=['yt-dlp', '-o', '/tmp/tmpslt2jd8f/EHl_eQhgefw___%(uploader)s___%(title)s___%(id)s___%(duration)d___.%(ext)s', '-x', '--audio-quality', '0', '--audio-format', 'wav', '--retries', '50', '--force-ipv4', '--ignore-errors', 'https://www.youtube.com/watch?v=-7lV0oJ0QXc'], returncode=0)

In [ ]:
download_song("EHl_eQhgefw")

CompletedProcess(args=['yt-dlp', '-o', 'song_output/%(uploader)s___%(title)s___%(id)s___%(duration)d___.%(ext)s', '-x', '--audio-quality', '5', '--audio-format', 'mp3', '--retries', '50', '--force-ipv4', '--ignore-errors', 'https://www.youtube.com/watch?v=EHl_eQhgefw'], returncode=0)

In [ ]:
glob.glob(os.path.join("song_output", "*.webm"))

[]

# download midi

In [10]:
multiprocessing.cpu_count()

24

In [ ]:
jobs = Parallel(n_jobs=16, prefer="threads")(
    delayed(download_midi)(
        piano_id=value["piano_ids"],
        song_id=value["pop_ids"],
    ) for value in tqdm(data_to_load.to_dict("records"))
)

In [13]:
len(os.listdir("song2midi/midi_output"))

855

In [28]:
!zip -r /content/midi.zip /content/midi_output

Segment 2 / 33
  adding: content/midi_output/ (stored 0%)
  adding: content/midi_output/F_74JqOubxk___Flow Music Piano___BTS (방탄소년단) - Airplane pt.2 Piano Tutorial 피아노 배우기___-F68f8ztrvw.mid (deflated 36%)
  adding: content/midi_output/hWYM5QEt0Fg___Piano in U___가호(Gaho) - 시작(Start) 이태원 클라쓰 OST(ITAEWON CLASS) ⧸ Piano cover ⧸ Sheet___-u5gsBF9wbY.mid (deflated 39%)
  adding: content/midi_output/5g4KsIizYhQ___PineappleChord___잔나비 (JANNABI) - 주저하는 연인들을 위해 (For Lovers Who Hesitate) 피아노 편곡 ｜ 피아노 악보 ｜ Piano Cover ｜ Piano Sheet___41DdcznIHSg.mid (deflated 34%)
  adding: content/midi_output/yFkvinwYbes___Piano in U___IU(아이유) _ eight(에잇) (Prod.&Feat. SUGA of BTS) ⧸ Piano Cover ⧸ Sheet___3NpWgnfOlwk.mid (deflated 35%)
  adding: content/midi_output/W4qsT-CFVdA___YourPianoCover___No Brainer - Justin Bieber, DJ Khaled, Quavo - Piano Cover Video by YourPianoCover___3DKGf_Us3MI.mid (deflated 36%)
  adding: content/midi_output/iqRkTGkAs9I___Piano in U___IU(아이유) _ Blueming(블루밍)⧸Piano cover⧸Sheet___0Rdmge

In [29]:
!cp /content/midi.zip /content/drive/MyDrive

Segment 48 / 55
Segment 11 / 33
Segment 49 / 55


In [ ]:
midi_outp = "midi_output"

unique_arranger = set()

for filename in os.listdir(midi_outp):
    if filename.endswith(".mid"):
        identifier = filename.split('___')[0]
        unique_arranger.add(identifier)

df = pd.DataFrame(list(unique_arranger), columns=['arranger_name'])

output_csv = "arrangers.csv"
df.to_csv(output_csv, index=False)

In [ ]:
!cp /content/arrangers.csv /content/drive/MyDrive

# download song

In [14]:
song_to_load = list(set(data_to_load.to_dict("list")["pop_ids"]))

In [15]:
len(song_to_load)

761

In [ ]:
jobs = Parallel(n_jobs=16, prefer="threads")(
    delayed(download_song)(
        song_id=value,
    ) for value in tqdm(song_to_load)
)

In [19]:
len(os.listdir("song2midi/song_output"))

715

In [ ]:
[download_song_by_link(process.args[-1]) for process in jobs if process.returncode != 0]

[CompletedProcess(args=['yt-dlp', '-o', 'song_output/%(uploader)s___%(title)s___%(id)s___%(duration)d___.%(ext)s', '-x', '--audio-quality', '5', '--audio-format', 'mp3', '--retries', '50', '--force-ipv4', '--ignore-errors', 'https://www.youtube.com/watch?v=KQIsk0ba5q0'], returncode=1),
 CompletedProcess(args=['yt-dlp', '-o', 'song_output/%(uploader)s___%(title)s___%(id)s___%(duration)d___.%(ext)s', '-x', '--audio-quality', '5', '--audio-format', 'mp3', '--retries', '50', '--force-ipv4', '--ignore-errors', 'https://www.youtube.com/watch?v=Xttlzf0RJwg'], returncode=0),
 CompletedProcess(args=['yt-dlp', '-o', 'song_output/%(uploader)s___%(title)s___%(id)s___%(duration)d___.%(ext)s', '-x', '--audio-quality', '5', '--audio-format', 'mp3', '--retries', '50', '--force-ipv4', '--ignore-errors', 'https://www.youtube.com/watch?v=H7qtectI-J4'], returncode=1),
 CompletedProcess(args=['yt-dlp', '-o', 'song_output/%(uploader)s___%(title)s___%(id)s___%(duration)d___.%(ext)s', '-x', '--audio-quality',

In [ ]:
!zip -r /content/song.zip /content/song_output

  adding: content/song_output/ (stored 0%)
  adding: content/song_output/Ratna Ariani___SUNG SI KYUNG (성시경) - TWO PEOPLE (두 사람) LYRICS 가사___59YLqazkQ9U___253___.mp3 (deflated 2%)
  adding: content/song_output/Lemoring___BTS - Miss Right (방탄소년단 - Miss Right) [Color Coded Lyrics⧸Han⧸Rom⧸Eng⧸가사]___gQEibb19w8k___243___.mp3 (deflated 1%)
  adding: content/song_output/Jaeguchi___BTOB(비투비) - 아름답고도 아프구나(Beautiful Pain) LYRICS (Color Coded Eng⧸Rom⧸Han⧸가사)___J57HXP79OjU___240___.mp3 (deflated 1%)
  adding: content/song_output/츄러스 ℂ𝕙𝕦𝕣𝕣𝕠𝕤___가사영상 ｜ 김건모 - 혼자만의 사랑___agCddlN1bIk___279___.mp3 (deflated 1%)
  adding: content/song_output/AlphaSubs___Paul Kim (폴킴) - Hangover (너도 아는) Lyrics⧸가사 [Han｜Rom｜Eng]___lArgiD9bpdk___230___.mp3 (deflated 1%)
  adding: content/song_output/뮤젯 MUSET___저스틴 비버 ｜ “넌 내가 사랑이라고 부를 단 한 사람이야” Justin Bieber - Anyone [가사해석]___seexiwp1da8___191___.mp3 (deflated 1%)
  adding: content/song_output/Kayten Park___Connie Talbot - Over The Rainbow (HQ) 영화 오즈의 마법사 OST 가사 ⧸ 해석___Ckn_QhOjU

In [ ]:
!cp /content/song.zip /content/drive/MyDrive